In [1]:
import torch
from transformers import AutoModelForCausalLM

# Load the model
model = AutoModelForCausalLM.from_pretrained("THUDM/cogagent-chat-hf")

# Explore the attributes
dir(model)

Please 'pip install apex'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_assisted_decoding',
 '_auto_class',
 '_autoset_attn_implementation',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_beam_sample',
 '_beam_search',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_check_and_enable_sdpa',
 '_compiled_call_impl',
 '_constrained_beam_search',
 '_contrastive_search',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_expand_inputs_for_generation',
 

In [2]:
# Print the top-level submodules
for name, module in model.named_children():
    print(f"Submodule: {name} - {module}")


Submodule: model - CogAgentModel(
  (embed_tokens): Embedding(32000, 4096, padding_idx=0)
  (layers): ModuleList(
    (0-31): 32 x CogAgentDecoderLayer(
      (self_attn): VisionExpertAttention(
        (rotary_emb): RotaryEmbedding()
        (vision_expert_query_key_value): Linear(in_features=4096, out_features=12288, bias=False)
        (vision_expert_dense): Linear(in_features=4096, out_features=4096, bias=False)
        (language_expert_query_key_value): Linear(in_features=4096, out_features=12288, bias=False)
        (language_expert_dense): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (cross_attn): CrossAttention(
        (query): Linear(in_features=4096, out_features=1024, bias=False)
        (key_value): Linear(in_features=1024, out_features=2048, bias=False)
        (dense): Linear(in_features=1024, out_features=4096, bias=False)
      )
      (mlp): VisionExpertMLP(
        (language_mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_featu

In [3]:
# Print the top-level submodules to identify possible visual components
print("Top-level submodules:")
for name, module in model.named_children():
    print(f"Submodule: {name} - {module}")

# Drill down into specific submodules if necessary
for name, module in model.named_children():
    print(f"\nInspecting submodule: {name}")
    for sub_name, sub_module in module.named_children():
        print(f"  Submodule: {sub_name} - {sub_module}")

Top-level submodules:
Submodule: model - CogAgentModel(
  (embed_tokens): Embedding(32000, 4096, padding_idx=0)
  (layers): ModuleList(
    (0-31): 32 x CogAgentDecoderLayer(
      (self_attn): VisionExpertAttention(
        (rotary_emb): RotaryEmbedding()
        (vision_expert_query_key_value): Linear(in_features=4096, out_features=12288, bias=False)
        (vision_expert_dense): Linear(in_features=4096, out_features=4096, bias=False)
        (language_expert_query_key_value): Linear(in_features=4096, out_features=12288, bias=False)
        (language_expert_dense): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (cross_attn): CrossAttention(
        (query): Linear(in_features=4096, out_features=1024, bias=False)
        (key_value): Linear(in_features=1024, out_features=2048, bias=False)
        (dense): Linear(in_features=1024, out_features=4096, bias=False)
      )
      (mlp): VisionExpertMLP(
        (language_mlp): MLP(
          (gate_proj): Linear(in_fe